# **Context Sensitivity: Improving Performance of Language Models Using Flash Ranking**


In the fast-evolving world of artificial intelligence, the performance of language models is a hot topic. With applications ranging from chatbots to content creation, these models are transforming how we interact with technology. A critical aspect of their efficiency is how quickly and accurately they can rank and retrieve information. Here is a standard representation for building a RAG system where we load the documents, extract text from them, store it in a vector database, and retrieve the information based on the query.

This problem highlights the need for more effective ranking mechanisms that can pinpoint and elevate crucial data, regardless of its position within a text. Flash Ranking addresses this by ensuring that even the most deeply embedded information is efficiently retrieved and utilized. This blog will delve into the mechanics of Flash Ranking, exploring its benefits and its impact on the performance of language models. Here is a more advanced technique for developing an RAG system using FalshRank.

A crucial element of RAG's efficiency lies in its ability to retrieve information swiftly and accurately. This is where Flash Ranking becomes essential. Flash Ranking is a novel method that improves the speed and precision of information retrieval in language models, overcoming the limitations of traditional ranking systems that often struggle with resource consumption and sluggishness. A concept closely related to this discussion is the "Lost in the Middle" problem, where vital information embedded within long passages often goes unnoticed by conventional retrieval systems. In the below chart, we observe that the models capture the context better at the beginning and at the end but in the middle, there is a dip in the performance. Here is a snapshot from the paper, Lost in the Middle: How Language Models Use Long Contexts

In [ ]:
# ! pip -q install tiktoken pypdf sentence-transformers InstructorEmbedding langchain_community huggingface_hub langchain-huggingface faiss-cpu flashrank

## Import the neccessary libraries

In [ ]:
import pandas as pd
import pickle
import faiss
import os
from langchain_huggingface import HuggingFaceEndpoint
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.vectorstores import FAISS
from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from flashrank.Ranker import Ranker, RerankRequest
import json

In [ ]:
from google.colab import userdata

userdata.get("HF_API_KEY")
userdata.get("HF_TOKEN")

print("API's loaded")

API's loaded


## Loading the data

In [ ]:
loader = DirectoryLoader(f"/content/data", glob="*.txt", loader_cls=TextLoader)
documents = loader.load()
documents

[Document(metadata={'source': '/content/data/Politics.txt'}, page_content='The G7 is a club of Western nations (with Japan given that status as an ally of the West and a major economy) that have dominated the world and its institutions, in some cases for centuries, and retain the ambition to maintain that position by policy coordination amongst themselves and by co-opting rising powers, including India, given the shifts in global power in recent decades.\n\nThe G7 recognised that they could not manage the 2008 financial crisis on their own and needed a wider international partnership, but one under their aegis. With this in mind, the G20 forum hitherto at the finance minister level was raised to the summit level. The G20 agenda is, however, shifting increasingly towards the interests and priorities of the developing countries (now being referred to as the Global South). During India’s G20 presidency, with India holding the Voice of the Global South summits before presiding over the G20

## Splitting the document using RecursiveCharacterTextSplitter

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=10)
texts = text_splitter.split_documents(documents)
texts

[Document(metadata={'source': '/content/data/Politics.txt'}, page_content='The G7 is a club of Western nations (with Japan given that status as an ally of the West and a major'),
 Document(metadata={'source': '/content/data/Politics.txt'}, page_content='a major economy) that have dominated the world and its institutions, in some cases for centuries,'),
 Document(metadata={'source': '/content/data/Politics.txt'}, page_content='and retain the ambition to maintain that position by policy coordination amongst themselves and by'),
 Document(metadata={'source': '/content/data/Politics.txt'}, page_content='and by co-opting rising powers, including India, given the shifts in global power in recent'),
 Document(metadata={'source': '/content/data/Politics.txt'}, page_content='in recent decades.'),
 Document(metadata={'source': '/content/data/Politics.txt'}, page_content='The G7 recognised that they could not manage the 2008 financial crisis on their own and needed a'),
 Document(metadata={'sourc

In [ ]:
# ! pip install sentence-transformers==2.2.2

In [ ]:
from langchain.embeddings import HuggingFaceInstructEmbeddings
instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl")

load INSTRUCTOR_Transformer
max_seq_length  512


## Storing the information in Vector database - FAISS

In [ ]:
import faiss
from langchain.vectorstores import FAISS
db_instructEmbedd = FAISS.from_documents(texts, instructor_embeddings)

retriever = db_instructEmbedd.as_retriever(search_kwargs={"k": 10})
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceInstructEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7eb2a7d61660>, search_kwargs={'k': 10})

## Testing the set up - Trail 1

In [ ]:
question = "Why are all the countries meeting and what is it about?"
doc = retriever.get_relevant_documents(question)
doc

[Document(metadata={'source': '/content/data/Politics.txt', 'relevance_score': 0.0019604184}, page_content='for promoting multipolarity, a greater role of developing countries in global governance, more'),
 Document(metadata={'source': '/content/data/Politics.txt', 'relevance_score': 0.94645005}, page_content='India holding the Voice of the Global South summits before presiding over the G20 and at the'),
 Document(metadata={'source': '/content/data/Politics.txt', 'relevance_score': 0.17093574}, page_content='countries (now being referred to as the Global South). During India’s G20 presidency, with India'),
 Document(metadata={'source': '/content/data/Politics.txt', 'relevance_score': 0.9507697}, page_content='permanent member at India’s initiative, the pro-Global South content of the G20 agenda has got'),
 Document(metadata={'source': '/content/data/Politics.txt', 'relevance_score': 0.0058038207}, page_content='needed a wider international partnership, but one under their aegis. With t

## Formatting the content and converting it to dataframe

In [ ]:
formatted_data = []
for idx, doc in enumerate(doc):
    entry = {
        "id": idx + 1,  # Start id from 1
        "text": doc.page_content,
        # "meta": doc.metadata
    }
    formatted_data.append(entry)

# Convert to JSON string
formatted_data_json = json.dumps(formatted_data, indent=3)
# print(formatted_data_json)

# Parse the JSON data
parsed_data = json.loads(formatted_data_json)

# # Extract id and text into lists
ids = [item['id'] for item in parsed_data]
texts = [item['text'] for item in parsed_data]

# # Create a DataFrame
df_top_n_retrievals = pd.DataFrame({'id': ids, 'text': texts})
df_top_n_retrievals




,id,text
0,1,"for promoting multipolarity, a greater role of..."
1,2,India holding the Voice of the Global South su...
2,3,countries (now being referred to as the Global...
3,4,"permanent member at India’s initiative, the pr..."
4,5,"needed a wider international partnership, but ..."
5,6,"more equity in international relations, and in..."
6,7,"at the conclusion of its work, and with the in..."
7,8,"on global issues. BRICS, a group of non-Wester..."
8,9,the G20 forum hitherto at the finance minister...
9,10,"Both the G7 and the G20, however, face challen..."


# Using FlashRank

In [ ]:
from flashrank import Ranker, RerankRequest
ranker = Ranker()

In [ ]:
rerankrequest = RerankRequest(query=question, passages=formatted_data)
flashrank_results = ranker.rerank(rerankrequest)
flashrank_results

[{'id': 8,
  'text': 'on global issues. BRICS, a group of non-Western countries, is getting expanded to resist the',
  'meta': {'source': '/content/data/Politics.txt'},
  'score': 0.00040620775},
 {'id': 3,
  'text': 'countries (now being referred to as the Global South). During India’s G20 presidency, with India',
  'meta': {'source': '/content/data/Politics.txt'},
  'score': 0.00038824725},
 {'id': 1,
  'text': 'for promoting multipolarity, a greater role of developing countries in global governance, more',
  'meta': {'source': '/content/data/Politics.txt'},
  'score': 4.8548136e-05},
 {'id': 6,
  'text': 'more equity in international relations, and introducing much-needed reforms in the international',
  'meta': {'source': '/content/data/Politics.txt'},
  'score': 4.1336738e-05},
 {'id': 5,
  'text': 'needed a wider international partnership, but one under their aegis. With this in mind, the G20',
  'meta': {'source': '/content/data/Politics.txt'},
  'score': 3.0409723e-05},
 {'id':

In [ ]:
df_re_rank = pd.DataFrame(flashrank_results)[['id', 'text']]
df_re_rank.rename(columns = {'id':'Re-rank-id', 'text': 'Re-rank-text'}, inplace = True)
df_re_rank

,Re-rank-id,Re-rank-text
0,8,"on global issues. BRICS, a group of non-Wester..."
1,3,countries (now being referred to as the Global...
2,1,"for promoting multipolarity, a greater role of..."
3,6,"more equity in international relations, and in..."
4,5,"needed a wider international partnership, but ..."
5,7,"at the conclusion of its work, and with the in..."
6,4,"permanent member at India’s initiative, the pr..."
7,2,India holding the Voice of the Global South su...
8,10,"Both the G7 and the G20, however, face challen..."
9,9,the G20 forum hitherto at the finance minister...


# Traditional approach Vs FlashRank approach

It is always interesting to examine the ranking of both approaches. Here are a few observations.
The traditional approach generated a response with 109 words and it answered our question. Most importantly the context was not lost and the response made sense.
The FashRank approach generated a shorter response with 83 words with context intact and a response much sharper.

In [ ]:
df_compare = pd.concat([df_top_n_retrievals, df_re_rank], axis = 1)
df_compare

,id,text,Re-rank-id,Re-rank-text
0,1,"for promoting multipolarity, a greater role of...",8,"on global issues. BRICS, a group of non-Wester..."
1,2,India holding the Voice of the Global South su...,3,countries (now being referred to as the Global...
2,3,countries (now being referred to as the Global...,1,"for promoting multipolarity, a greater role of..."
3,4,"permanent member at India’s initiative, the pr...",6,"more equity in international relations, and in..."
4,5,"needed a wider international partnership, but ...",5,"needed a wider international partnership, but ..."
5,6,"more equity in international relations, and in...",7,"at the conclusion of its work, and with the in..."
6,7,"at the conclusion of its work, and with the in...",4,"permanent member at India’s initiative, the pr..."
7,8,"on global issues. BRICS, a group of non-Wester...",2,India holding the Voice of the Global South su...
8,9,the G20 forum hitherto at the finance minister...,10,"Both the G7 and the G20, however, face challen..."
9,10,"Both the G7 and the G20, however, face challen...",9,the G20 forum hitherto at the finance minister...


In [ ]:
# def highlight_first_five_rows(s):
#     if s.name < 5:  # Apply style to the first 5 rows
#         return ['background-color: lightgreen'] * len(s)
#     else:
#         return [''] * len(s)

# # Apply the styling
# df_compare = df_compare.style.apply(highlight_first_five_rows, axis=1)

# # Display the styled DataFrame
# df_compare
df_compare['Re-rank-text'][8]

'Both the G7 and the G20, however, face challenges from other platforms for consensus-building on'

# Generating the LLM output from the traditional version

In [ ]:
from langchain_core.runnables import RunnablePassthrough

In [ ]:
repo_id="mistralai/Mistral-7B-Instruct-v0.2"
llm=HuggingFaceEndpoint(repo_id=repo_id,max_length=128,
                                temperature=0.7,token='HF_TOKEN')

template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

# Create a prompt from the template
prompt = ChatPromptTemplate.from_template(template)


chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.
                    token was transferred to model_kwargs.
                    Please make sure that token is what you intended.


In [ ]:
response  = chain.invoke(question)
print(response)


Answer: The countries are meeting within the context of the G20 forum to promote multipolarity, expand the role of developing countries in global governance, and address global issues. During India's G20 presidency, efforts were made to introduce more equity in international relations and reforms in the international system. The G20 agenda has needed a wider international partnership, and one under the leadership of the Global South. The G20 summit level was raised from the finance minister level to address these matters more effectively. Additionally, the African Union was included as a G20 permanent observer. BRICS, a group of non-Western countries, is expanding within the G20 to resist other consensus-building platforms.


# Generating the LLM output from FlashRank

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank
from langchain.chains import RetrievalQA

compressor = FlashrankRerank(top_n = 10)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)


In [ ]:
chain_reranked = RetrievalQA.from_chain_type(llm=llm, retriever=compression_retriever)

In [ ]:
compressed_docs = compression_retriever.invoke(
    question
)
compressed_docs

[Document(metadata={'source': '/content/data/Politics.txt', 'relevance_score': 0.9965717}, page_content='on global issues. BRICS, a group of non-Western countries, is getting expanded to resist the'),
 Document(metadata={'source': '/content/data/Politics.txt', 'relevance_score': 0.9927882}, page_content='Both the G7 and the G20, however, face challenges from other platforms for consensus-building on'),
 Document(metadata={'source': '/content/data/Politics.txt', 'relevance_score': 0.9770818}, page_content='at the conclusion of its work, and with the inclusion of the African Union as a G20 permanent'),
 Document(metadata={'source': '/content/data/Politics.txt', 'relevance_score': 0.9507697}, page_content='permanent member at India’s initiative, the pro-Global South content of the G20 agenda has got'),
 Document(metadata={'source': '/content/data/Politics.txt', 'relevance_score': 0.94645005}, page_content='India holding the Voice of the Global South summits before presiding over the G20 a

In [ ]:
print([doc.page_content for doc in compressed_docs])
# len(compressed_docs)

['on global issues. BRICS, a group of non-Western countries, is getting expanded to resist the', 'Both the G7 and the G20, however, face challenges from other platforms for consensus-building on', 'at the conclusion of its work, and with the inclusion of the African Union as a G20 permanent', 'permanent member at India’s initiative, the pro-Global South content of the G20 agenda has got', 'India holding the Voice of the Global South summits before presiding over the G20 and at the', 'countries (now being referred to as the Global South). During India’s G20 presidency, with India', 'needed a wider international partnership, but one under their aegis. With this in mind, the G20', 'the G20 forum hitherto at the finance minister level was raised to the summit level. The G20 agenda', 'more equity in international relations, and introducing much-needed reforms in the international', 'for promoting multipolarity, a greater role of developing countries in global governance, more']


In [ ]:
chain_reranked = RetrievalQA.from_chain_type(llm=llm, retriever=compression_retriever)

In [ ]:
chain_reranked.invoke(question)

{'query': 'Why are all the countries meeting and what is it about?',
 'result': ' The countries, including those in the Global South, are meeting in the context of the G20 and BRICS forums. The G20 is getting expanded to resist the dominance of Western countries and promote multipolarity. The G20 agenda now includes more equity in international relations and a greater role for developing countries in global governance. India, as a member of both forums, is holding the Voice of the Global South summits and raising the G20 forum to the summit level to address these issues.'}

In [ ]:
formatted_data = []
for idx, doc in enumerate(compressed_docs):
    entry = {
        "id": idx + 1,  # Start id from 1
        "text": doc.page_content,
        # "meta": doc.metadata
    }
    formatted_data.append(entry)

# Convert to JSON string
formatted_data_json = json.dumps(formatted_data, indent=3)
# print(formatted_data_json)

# Parse the JSON data
parsed_data = json.loads(formatted_data_json)

# # Extract id and text into lists
ids = [item['id'] for item in parsed_data]
texts = [item['text'] for item in parsed_data]

# # Create a DataFrame
df_retrievals_compressed = pd.DataFrame({'id': ids, 'text': texts})
df_retrievals_compressed

,id,text
0,1,"on global issues. BRICS, a group of non-Wester..."
1,2,"Both the G7 and the G20, however, face challen..."
2,3,"at the conclusion of its work, and with the in..."
3,4,"permanent member at India’s initiative, the pr..."
4,5,India holding the Voice of the Global South su...
5,6,countries (now being referred to as the Global...
6,7,"needed a wider international partnership, but ..."
7,8,the G20 forum hitherto at the finance minister...
8,9,"more equity in international relations, and in..."
9,10,"for promoting multipolarity, a greater role of..."


# Comparision of ranks of the retrieved chunks

It is always interesting to examine the ranking of both approaches. Here are a few observations.


*   The traditional approach generated a response with 109 words and it answered our question. Most importantly the context was not lost and the response made sense.
*   The FashRank approach generated a shorter response with 83 words with context intact and a response much sharper.



In [ ]:
merged_df = pd.merge(df_retrievals_compressed, df_top_n_retrievals[['id', 'text']], on='text', how='left', suffixes=('', '_new'))
df_retrievals_compressed['id'] = merged_df['id_new'].combine_first(df_retrievals_compressed['id'])
df_compare_final = pd.concat([df_compare, df_retrievals_compressed], axis = 1)
df_compare_final

,id,text,Re-rank-id,Re-rank-text,id,text
0,1,"for promoting multipolarity, a greater role of...",8,"on global issues. BRICS, a group of non-Wester...",8,"on global issues. BRICS, a group of non-Wester..."
1,2,India holding the Voice of the Global South su...,3,countries (now being referred to as the Global...,10,"Both the G7 and the G20, however, face challen..."
2,3,countries (now being referred to as the Global...,1,"for promoting multipolarity, a greater role of...",7,"at the conclusion of its work, and with the in..."
3,4,"permanent member at India’s initiative, the pr...",6,"more equity in international relations, and in...",4,"permanent member at India’s initiative, the pr..."
4,5,"needed a wider international partnership, but ...",5,"needed a wider international partnership, but ...",2,India holding the Voice of the Global South su...
5,6,"more equity in international relations, and in...",7,"at the conclusion of its work, and with the in...",3,countries (now being referred to as the Global...
6,7,"at the conclusion of its work, and with the in...",4,"permanent member at India’s initiative, the pr...",5,"needed a wider international partnership, but ..."
7,8,"on global issues. BRICS, a group of non-Wester...",2,India holding the Voice of the Global South su...,9,the G20 forum hitherto at the finance minister...
8,9,the G20 forum hitherto at the finance minister...,10,"Both the G7 and the G20, however, face challen...",6,"more equity in international relations, and in..."
9,10,"Both the G7 and the G20, however, face challen...",9,the G20 forum hitherto at the finance minister...,1,"for promoting multipolarity, a greater role of..."


In [ ]:
df_compare_final.iloc[:,[0,1,4,5]]

,id,text,id,text
0,1,"for promoting multipolarity, a greater role of...",8,"on global issues. BRICS, a group of non-Wester..."
1,2,India holding the Voice of the Global South su...,10,"Both the G7 and the G20, however, face challen..."
2,3,countries (now being referred to as the Global...,7,"at the conclusion of its work, and with the in..."
3,4,"permanent member at India’s initiative, the pr...",4,"permanent member at India’s initiative, the pr..."
4,5,"needed a wider international partnership, but ...",2,India holding the Voice of the Global South su...
5,6,"more equity in international relations, and in...",3,countries (now being referred to as the Global...
6,7,"at the conclusion of its work, and with the in...",5,"needed a wider international partnership, but ..."
7,8,"on global issues. BRICS, a group of non-Wester...",9,the G20 forum hitherto at the finance minister...
8,9,the G20 forum hitherto at the finance minister...,6,"more equity in international relations, and in..."
9,10,"Both the G7 and the G20, however, face challen...",1,"for promoting multipolarity, a greater role of..."


## Conclusion
In conclusion, FlashRank is a technique that improves language model performance by prioritizing and refining data retrieval, enhancing the final output of language models. This approach addresses challenges like slow information retrieval and overlooked middle segments of text chunks, ensuring the effective utilization of crucial information.